In [1]:
import torch
import torchtext
import pandas as pd
import gensim
import gensim.downloader as api
from torchtext.vocab import Vectors

In [2]:
df_train = pd.read_csv('data/processed_train.csv') #TRAIN DATA
#UNCOMMENT BELOW WHEN RUNNING FOR THE FIRST TIME
#wv = api.load('word2vec-google-news-300') #DOWNLOAD WORD VECTORS
#wv.save_word2vec_format('initial_embeddings.kv') #SAVE WORD VECTORS LOCALLY
vectors = Vectors(name='initial_embeddings.kv')

In [3]:
TEXT = torchtext.data.Field(
            pad_first=True,
            batch_first=False #CHANGED
        )

LABEL = torchtext.data.LabelField(dtype=torch.float)
datafields = [('Sentiment', LABEL), ('SentimentText', TEXT)]

train, val, test = torchtext.data.TabularDataset.splits(path='data/',
                                                  train='processed_train.csv',
                                                  validation='processed_val.csv',
                                                  test='processed_test.csv',
                                                  format='csv',
                                                  skip_header=True,
                                                  fields=datafields)

TEXT.build_vocab(
            train,
            max_size=1000,
            min_freq=5,
            vectors=vectors
)

LABEL.build_vocab(train,)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, val_iterator, test_iterator = torchtext.data.BucketIterator.splits(
                                (train, val, test),
                                batch_size=64,
                                sort_key=lambda x: len(x.SentimentText),
                                sort_within_batch=False,
                                device=device)

In [5]:
import torch.nn as nn

class GRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim,
                output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim).from_pretrained(TEXT.vocab.vectors)
        self.gru = nn.GRU(embedding_dim, 
                          hidden_dim,
                         num_layers=n_layers,
                         bidirectional=bidirectional,
                         dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, text):
        embedded_text = self.dropout(self.embedding(text))
        output, hidden = self.gru(embedded_text)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        x = self.fc(hidden.squeeze(0))
        return x

In [6]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = GRU(vocab_size=INPUT_DIM, 
            embedding_dim=EMBEDDING_DIM, 
            hidden_dim=HIDDEN_DIM, 
            output_dim=OUTPUT_DIM, 
            n_layers=2,
            bidirectional=True,
            dropout=0.1)

In [7]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [8]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [9]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.SentimentText).squeeze(1)
        
        loss = criterion(predictions, batch.Sentiment)
        
        acc = binary_accuracy(predictions, batch.Sentiment)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return model, epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            # print(batch.SentimentText)
            if batch.SentimentText.nelement() > 0:
                predictions = model(batch.SentimentText).squeeze(1)

                loss = criterion(predictions, batch.Sentiment)

                acc = binary_accuracy(predictions, batch.Sentiment)

                epoch_loss += loss.item()
                epoch_acc += acc.item()
            # else:
              #  print(f"Found a non-empty Tensorlist {batch.SentimentText}")
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [10]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [11]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    model, train_loss, train_acc  = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'gru_simple_.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

KeyboardInterrupt: 